In [4]:
#from promts import Promt
from dotenv import load_dotenv
load_dotenv()
import openai
import os

class Promt:
    

    params_dict = {"role" : ["system", "assistant", "user"], # Роль в промте
              "is_open": ["open", "close"], # открытые вопросы или с вариантами ответов
              "prof": ["Разработчик", "DevОps", "Сисадмин", "Аналитик", "Тестировщик", "Руководитель проекта"],
              "section": ["general","fronted","backend", "full stack", "QA manual","QA automat"],
              "technology": ["Python", "C++", "C#", "Java", "Java Script", "1C"], #потом добавлю и подумаю над структурой
              "level": ["junior", "junior+", "middle", "middle+", "senior"],
              "subtech": ["framework","lib","linked_tech"],
              "num_questions":""           
            }
    

   
    promt_path = {  ("system","open"): "promt_system_open.txt",
                    ("assistant","open"): "promt_assistant_open.txt",
                    ("user","open"): "promt_user_open.txt",
                    ("system","close"): "promt_system_close.txt",
                    ("assistant","close"): "promt_assistant_close.txt",
                    ("user","close"): "promt_user_open.txt"                  
                }
    
    replace_dict = {"Разработчик":"Программист, разработчик программного обеспечения на языке программирования",
                    "Аналитик":"аналитик, системный аналитик",
                    "Сисадмин": "системный админинистратор, системный инженер",
                    "Тестировщик":"тестировщик программного обеспечения, QA",
                    "manager":"manager",
                    "general": "общие"
                    }
    
    add_dict = {"open": '''Вопрос должен проверять не набор справочных знаний, а выявлять понимание подходов к решению тех или иных задач.
Вопрос должен быть таким, чтобы ответ специалиста мог продемострировать его практический опыт.'''
                }
    
    
    
    def get_base_promt(self, params:list):  #получить основной промт
    
    
        key_path = tuple(params[:2])
        promt_file_name = "Promts/"+Promt.promt_path[key_path]
        
        all_lists = []
        for lists in Promt.params_dict.values():  
            all_lists += lists  
        if len (all_lists) > len(set(all_lists)):  
            raise Exception ('Вcе значения  словаря  должны быть уникальными')
        
        if len (params) > len(set(params)):
            raise Exception ('Вcе значения параметров функции должны быть уникальными')
                    
        try:    
            with open (promt_file_name, 'r') as promt_file:
                promt_content = promt_file.read()
                
        except FileNotFoundError:
            print (f"Файл промта {promt_file_name} не найден")    
        
        for i, param_keys in enumerate(Promt.params_dict.keys()):
            param_value = params[i]  
            
            #если существует значение (расшифровка) в словаре параметров берем его для подстановкив промт, если нет, подставляем ключ
            if param_value in Promt.replace_dict: 
                replace_str = Promt.replace_dict[param_value] 
            else:
                replace_str = param_value  # иначе в подстановку сохраняем значение ключа replace_dict, оно же value в params
    
            promt_content = promt_content.replace("{"+param_keys+"}", replace_str)
            
            if param_value in Promt.add_dict:
                promt_content += Promt.add_dict[param_value]

        promt_file.close()
        
        return promt_content
    
    def get_promt_ext(self, params, promt_history):  #получить доп промт
        return ""  

openai.api_key = os.environ.get("OPENAI_API_KEY")

client = openai.OpenAI()
cur_promt = Promt()

params = ["open", "dev","general", "Python", "middle", "","20"]
system_promt = cur_promt.get_base_promt(["system"] + params)
assistant_promt = cur_promt.get_base_promt(["assistant"]  + params)
user_promt = cur_promt.get_base_promt(["user"]+ params)

response = client.chat.completions.create(
    model="gpt-4o",
    temperature = 0.1,
    messages=[
        {"role": "system", "content": system_promt},
        {'role':'assistant', 'content': assistant_promt},
        {"role": "user", "content": user_promt}
    ]
)
# сгенерированный ответ
generated_response = response.choices[0].message.content

print(f"\nВопросы: {generated_response}")


Вопросы: ### Общие вопросы для программистов и разработчиков программного обеспечения на языке Python (уровень middle)

#### Основы языка Python
1. Как вы подходите к выбору между использованием списков и кортежей в Python?
2. В каких случаях вы предпочитаете использовать генераторы списков вместо обычных циклов?
3. Как вы реализуете обработку исключений в Python и в каких случаях это особенно важно?
4. Как вы используете декораторы в Python и для каких задач они наиболее полезны?
5. Как вы подходите к написанию и использованию контекстных менеджеров?

#### Работа с данными
6. Как вы работаете с большими объемами данных в Python и какие библиотеки используете для этого?
7. Как вы оптимизируете работу с файлами в Python?
8. Как вы подходите к обработке и анализу данных с использованием библиотеки Pandas?
9. Как вы используете библиотеку NumPy для работы с массивами данных?
10. Как вы реализуете и оптимизируете работу с базами данных в Python?

#### Объектно-ориентированное программиров

In [3]:
params = ["open", "dev","general", "Python", "middle", "","20"]
print(["system"] + params)

['system', 'open', 'dev', 'general', 'Python', 'middle', '', '20']
